[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [2]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

In [186]:
client_key = "fill in your key here"
client_secret = "fill in your secret here" 

In [10]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [11]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [12]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [13]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [14]:
access_token = json_data['access_token']

# Get Tweets
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [142]:
max_tweets=1000 #CHANGED TO 200 FOR TESTING - MUST BE at least 1000 for analysis
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [143]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","DaBears","Eagles", "Cowboys","Raiders", "ChiefsKingdom"]

for keyword in keywords:
    #parameters popular and extended tweet mode
    parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit, 
                     'tweet_mode': 'extended'}

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit,
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1329433901744451584
0 100
searching tweets with id  < 1329363684821372928
1 100
searching tweets with id  < 1329265253721526273
2 100


As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

In [187]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 6000


# Let's print some tweets

In [145]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [146]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Nov 19 14:38:36 +0000 2020',
  'display_text_range': [0, 165],
  'entities': {'hashtags': [],
               'media': [{'display_url': 'pic.twitter.com/RRmWQqNdBd',
                          'expanded_url': 'https://twitter.com/RyanDeto/status/1329433901744451584/photo/1',
                          'id': 1329433894324760577,
                          'id_str': '1329433894324760577',
                          'indices': [166, 189],
                          'media_url': 'http://pbs.twimg.com/media/EnMZFU-XcAEXDTW.jpg',
                          'media_url_https': 'https://pbs.twimg.com/media/EnMZFU-XcAEXDTW.jpg',
                          'sizes': {'large': {'h': 801,
                                              'resize': 'fit',
                                              'w': 1242},
                                    'medium': {'h': 774,
                                               'resize': 'fit',
              

In [147]:
for status in tweets[0:max_tweet_to_print]:
    print(status['full_text'] + '\n')

The British Financial Times not understanding exactly where and what Sheetz is, only outdone by its meekness in describing Pittburghers’ hatred of the Dallas Cowboys https://t.co/RRmWQqNdBd

Andy Dalton, who is now ready to start Sunday vs. the Vikings, said the coronavirus hit him hard:

https://t.co/3ZgsJNxEbj

Cowboys' Andy Dalton Says He Still Can't Taste, Smell After COVID Diagnosis https://t.co/BMfLO1H6MP

AS IF WE NEEDED THE COWBOYS TO BREAK OUR HEARTS AGAIN https://t.co/1lnuSE676e



# Pandas DataFrame for storing tweet feed


# Printing keys in tweets

In [148]:
print(tweets[1].keys())
print(len(tweets[1].keys()))

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'team'])
27


In [149]:
import pandas as pd

In [150]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       

In [151]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Thu Nov 19 14:38:36 +0000 2020',
  'id': 1329433901744451584,
  'id_str': '1329433901744451584',
  'full_text': 'The British Financial Times not understanding exactly where and what Sheetz is, only outdone by its meekness in describing Pittburghers’ hatred of the Dallas Cowboys https://t.co/RRmWQqNdBd',
  'truncated': False,
  'display_text_range': [0, 165],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1329433894324760577,
     'id_str': '1329433894324760577',
     'indices': [166, 189],
     'media_url': 'http://pbs.twimg.com/media/EnMZFU-XcAEXDTW.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EnMZFU-XcAEXDTW.jpg',
     'url': 'https://t.co/RRmWQqNdBd',
     'display_url': 'pic.twitter.com/RRmWQqNdBd',
     'expanded_url': 'https://twitter.com/RyanDeto/status/1329433901744451584/photo/1',
     'type': 'photo',
     'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
      'small': {'w': 

In [152]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Tweet Cleaning

In [153]:
print("Rows with either null text or teams: {}".format(football_feed_df[["text", "team"]].isnull().any(axis = 1).sum())) 

Rows with either null text or teams: 100


In [154]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [155]:
football_feed_df[["full_text","team", "text"]].head()

,full_text,team,text
0,The British Financial Times not understanding ...,Dallas Cowboys,NaN
1,"Andy Dalton, who is now ready to start Sunday ...",Dallas Cowboys,NaN
2,Cowboys' Andy Dalton Says He Still Can't Taste...,Dallas Cowboys,NaN
3,AS IF WE NEEDED THE COWBOYS TO BREAK OUR HEART...,Dallas Cowboys,NaN
4,"@KarlSafchick My boys! And don’t worry, as a ...",Dallas Cowboys,NaN


In [156]:
football_feed_df.full_text.fillna(football_feed_df.text, inplace=True)
football_feed_df["full_text"] = football_feed_df["full_text"].apply(str)
football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,The British Financial Times not understanding ...,Dallas Cowboys,NaN
1,"Andy Dalton, who is now ready to start Sunday ...",Dallas Cowboys,NaN
2,Cowboys' Andy Dalton Says He Still Can't Taste...,Dallas Cowboys,NaN
3,AS IF WE NEEDED THE COWBOYS TO BREAK OUR HEART...,Dallas Cowboys,NaN
4,"@KarlSafchick My boys! And don’t worry, as a ...",Dallas Cowboys,NaN
...,...,...,...
395,@JosephPatrice I really just want to Spam this...,Dallas Cowboys,@JosephPatrice I really just want to Spam this...
396,RT @keilanadezignz: 🏈 Dallas Cowboys 2PC Tray ...,Dallas Cowboys,RT @keilanadezignz: 🏈 Dallas Cowboys 2PC Tray ...
397,RT @apa1906NETwork: Brother Charles Lewis Hale...,Dallas Cowboys,RT @apa1906NETwork: Brother Charles Lewis Hale...
398,RT @WilliamJoy: New: 8 people who tested posit...,Dallas Cowboys,RT @WilliamJoy: New: 8 people who tested posit...


In [157]:
print("Rows with either null text or teams: {}".format(football_feed_df[["full_text", "team"]].isnull().any(axis = 1).sum())) 

Rows with either null text or teams: 0


In [158]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')

In [159]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,lang,team,user_id,quoted_status_id,quoted_status_id_str,quoted_status,retweeted_status_id,text,hashtags,user_mentions
0,Thu Nov 19 14:38:36 +0000 2020,1329433901744451584,1329433901744451584,The British Financial Times not understanding ...,False,"[0, 165]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1329433894324760577, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Dallas Cowboys,355823359,NaN,NaN,NaN,NaN,NaN,[],
1,Fri Nov 20 13:05:43 +0000 2020,1329772914091159557,1329772914091159557,"Andy Dalton, who is now ready to start Sunday ...",False,"[0, 122]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Dallas Cowboys,51263592,NaN,NaN,NaN,NaN,NaN,[],
2,Thu Nov 19 21:20:33 +0000 2020,1329535054201044994,1329535054201044994,Cowboys' Andy Dalton Says He Still Can't Taste...,False,"[0, 99]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",...,en,Dallas Cowboys,16331010,NaN,NaN,NaN,NaN,NaN,[],
3,Sat Nov 21 02:40:58 +0000 2020,1329978079134052354,1329978079134052354,AS IF WE NEEDED THE COWBOYS TO BREAK OUR HEART...,False,"[0, 53]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Dallas Cowboys,750098054900953088,1.329970e+18,1329969772944093185,{'created_at': 'Sat Nov 21 02:07:58 +0000 2020...,NaN,NaN,[],
4,Sat Nov 21 02:40:08 +0000 2020,1329977868571406337,1329977868571406337,"@KarlSafchick My boys! And don’t worry, as a ...",False,"[14, 99]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1329977865316626434, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/#!/download/ipad"" ...",...,en,Dallas Cowboys,1035807834,NaN,NaN,NaN,NaN,NaN,[],Karl Safchick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Thu Nov 19 00:53:29 +0000 2020,1329226253904322565,1329226253904322565,@JosephPatrice I really just want to Spam this...,True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Dallas Cowboys,427951378,NaN,NaN,NaN,NaN,@JosephPatrice I really just want to Spam this...,[],Joe Patrice
396,Thu Nov 19 00:50:13 +0000 2020,1329225430109466624,1329225430109466624,RT @keilanadezignz: 🏈 Dallas Cowboys 2PC Tray ...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1329189406679318528, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,en,Dallas Cowboys,963930258184572928,NaN,NaN,NaN,1.329189e+18,RT @keilanadezignz: 🏈 Dallas Cowboys 2PC Tray ...,[],keilanadezignz 💕
397,Thu Nov 19 00:49:01 +0000 2020,1329225129323343874,1329225129323343874,RT @apa1906NETwork: Brother Charles Lewis Hale...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Dallas Cowboys,3302680656,NaN,NaN,NaN,1.329198e+18,RT @apa1906NETwork: Brother Charles Lewis Hale...,[],"Alpha Phi Alpha Fraternity, Inc. ®"
398,Thu Nov 19 00:48:46 +0000 2020,1329225065678954497,1329225065678954497,RT @WilliamJoy: New: 8 people who tested posit...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,en,Dallas Cowboys,168360685,NaN,NaN,NaN,1.329177e+18,RT @WilliamJoy:

In [160]:
def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

football_feed_df["full_text"] = football_feed_df["full_text"].apply(clean_tweet)

In [161]:
def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["full_text"].apply(emoji_retriever)

In [182]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["full_text"].apply(preprocess_text)

In [185]:
# DataFrame to csv

football_feed_df.to_csv('football_feed_df.csv')

In [183]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Team Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [184]:
sentiments

team
Bears      0.119359
Chiefs     0.131724
Cowboys    0.108805
Eagles     0.140848
Packers    0.117030
Raiders    0.135659
dtype: float64